## 8.2 Dictionaries

Python's `dict` class implements a restricted form of maps.
In M269, 'dictionary' (without any further qualification)
refers to an object of type `dict`.

<div class="alert alert-info">
<strong>Info:</strong> TM112 introduces Python dictionaries in Block&nbsp;3 Section&nbsp;2.1.
Some texts use 'dictionary' as a synonym for 'map'.
</div>

The operations are written in Python as follows, using the familiar list
notation, but using keys instead of 0, 1, 2, etc. as 'indices':

Operation | Python
:-|:-
new  |  `d = dict()`
size  |  `len(d)`
membership  |  `key in d`
associate  |  `d[key] = value`
lookup |  `d[key]`
delete  | `d.pop(key)`

Like for lists, the `pop` method returns the associated value,
and the negation of membership can be written `key not in d`.

We can represent the bilingual dictionary with a Python dictionary
in which the keys are strings and the values are lists of strings.

In [1]:
pt_to_en = dict()   # Portuguese to English dictionary
pt_to_en['alface'] = ['lattice']
pt_to_en['alface'] = ['lettuce']    # replace wrong entry
pt_to_en['carro'] = ['car']
pt_to_en['andar'] = ['floor', 'walk']
'carro' in pt_to_en

True

Dictionaries are iterable.

In [2]:
for word in pt_to_en:   # iterate over the keys
    for translation in pt_to_en[word]:
        print(word, 'means', translation)

alface means lettuce
carro means car
andar means floor
andar means walk


Python's implementation of maps guarantees that keys are iterated
in the same order they were added or last updated,
but you shouldn't rely on that in your M269 algorithms
to keep them working with any implementation of the map ADT.

The `items` method returns a list-like object of tuples,
one for each key–value pair. It's mostly used in for-loops.

In [3]:
for pair in pt_to_en.items():
    word = pair[0]
    for translation in pair[1]:
        print(word, 'means', translation)

alface means lettuce
carro means car
andar means floor
andar means walk


Python allows the following shorthand notation.

In [4]:
for (word, translations) in pt_to_en.items():
    for translation in translations:
        print(word, 'means', translation)

alface means lettuce
carro means car
andar means floor
andar means walk


Dictionary literals are written as comma-separated pairs within curly braces.
A colon separates each key from the corresponding value.
Here's a shorter way of defining the bilingual dictionary.

In [5]:
pt_to_en = {
    'alface': ['lettuce'],
    'carro': ['car'],
    'andar': ['floor', 'walk']
}

<div class="alert alert-warning">
<strong>Note:</strong> The empty dictionary can be written as <code>{}</code>, but in M269 we use <code>dict()</code>
instead, to avoid confusion with another data type, to be introduced later.
</div>

Dictionary keys may be integers that, unlike list indices,
don't have to be consecutive.
Here's a dictionary of addresses.
The keys are the house numbers; the values are the residents' names.

In [6]:
our_houses = {23: 'Alice', 45: 'Bob'}

We can check if two dictionaries have the same key–value pairs or not
with the (in)equality operators.
In a dictionary, the key–value pairs are in no particular order.

In [7]:
our_street = {45: 'Bob', 23: 'Alice'}
our_street == our_houses

True

In [8]:
our_street != {45: 'Bob', 23: 'Alissa'}

True

Like sequences, maps may be nested, i.e.
the value associated to a key may be a map.
For the bilingual dictionary, this could be used to distinguish the
meanings of a word.
For example, if 'andar' is used as a noun, then its translation is 'floor', whereas if 'andar' is used as a verb, then its translation is 'walk'.

In [9]:
pt2en = {
    'alface': {'noun': 'lettuce'},
    'carro': {'noun': 'car'},
    'andar': {'noun': 'floor', 'verb': 'walk'}
}

We access inner dictionary values in the same way as nested list items.

In [10]:
inner_dictionary = pt2en['andar']
print(inner_dictionary['verb'])
print(pt2en['andar']['verb'])   # shorter alternative

walk
walk


### 8.2.1 Mistakes

Accessing or deleting a non-existent key raises a **key error**.

In [11]:
pt_to_en['car']     # 'car' is among the values, not the keys

KeyError: 'car'

Dictionaries only retrieve data and check membership by key, not by value.

In [12]:
'carro' in pt_to_en

True

In [13]:
'car' in pt_to_en

False

Keys can't be added or removed while iterating over a dictionary.

In [14]:
roman_to_arabic = {'I': 1, 'V': 5, 'X': 10, 'L': 50}
for (key, value) in roman_to_arabic.items():
    roman_to_arabic[key+'I'] = value+1

RuntimeError: dictionary changed size during iteration

You may however change the values.

In [15]:
stock = {'trousers': 5, 't-shirt': 20, 'dress': 12}
for (key, value) in stock.items():
    stock[key] = 0      # all sold out
stock

{'trousers': 0, 't-shirt': 0, 'dress': 0}

Dictionaries implement a restricted map ADT:
keys can only be of types for which there's a hash function
and that doesn't include lists and dictionaries. For example, consider a map of
office building pairs to the names of their occupants. The keys can't be lists:

In [16]:
employee_by_location = {    # occupants of each building's offices
    ['Main building', 4]: ['Alice', 'Chan'],
    ['Annex', 3]: ['Bob']
}

TypeError: unhashable type: 'list'

We get a type error: lists are unhashable and thus the wrong type of key.
The keys must be tuples:

In [17]:
employee_by_location = {
    ('Main building', 4): ['Alice', 'Chan'],
    ('Annex', 3): ['Bob']
}

Using a list or a dictionary as part of a key also leads to a key error.
For example, `('Bob', [1, 'Jan', 1970])` and
`('Bob', {'day': 1, 'month: '1', 'year': 1970})` aren't valid Python keys, but
`('Bob', '1 Jan 1970')` and `('Bob', (1, 1, 1970))` are.
Fortunately, most applications of dictionaries don't need complicated keys:
integers, strings or tuples of both will suffice.

I explain in the next section why tuples are hashable but why lists aren't.

#### Exercise 8.2.1

Why can't we create a bilingual dictionary like this?

In [18]:
bilingual = dict()
bilingual['alface'] = 'lettuce'
bilingual['carro'] = 'car'
bilingual['andar'] = 'floor'
bilingual['andar'] = 'walk'

_Write your answer here._

[Answer](../32_Answers/Answers_08_2_01.ipynb)

### 8.2.2 Using dictionaries

To further illustrate the dictionary operations, let's consider the problem of
inverting a map, i.e. swapping keys and values, for bilingual dictionaries.

**Function**: invert\
**Inputs**: _original_, a map with strings as keys and sequences of strings as values\
**Preconditions**: true\
**Output**: _inverted_, a map with strings as keys and sequences of strings as values\
**Postconditions:** _inverted_(_word_) has _translation_ if and only if
_original_(_translation_) has _word_

The postconditions state that string _a_ translates to string _b_ in
the inverted map if and only if _b_ translates to _a_ in the original map.

For testing I will use an empty map (edge case) and
the Portuguese–English dictionary.
The inversion of the former is the empty map; the inversion of the latter is:

Key (English) | Value (Portuguese)
:-|:-
'lettuce'  |  ('alface')
'car'  |  ('carro')
'walk'  |  ('andar')
'floor'  |  ('andar')

Unfortunately, this isn't a very good test because the inverted dictionary
doesn't have multiple Portuguese translations for the same English word.
Let's add another translation of 'floor': 'chão'.
Since I have to change the Portuguese–English dictionary anyhow,
I add an edge case for the value: an empty sequence,
i.e. I add a Portuguese word but no English translation.
Here are the two new dictionaries: the input and the expected output.

Key (Portuguese) | Value (English) | Key (English) | Value (Portuguese)
:-|:-|-:|:-
'alface' | ('lettuce') | 'lettuce'  |  ('alface')
'carro' | ('car') | 'car'  |  ('carro')
'andar' | ('floor', 'walk') | 'walk'  |  ('andar')
'chão'  | ('floor')  | 'floor'  |  ('andar', 'chão')
'saudade'  | ()  |   |

#### Exercise 8.2.2

Describe an algorithm that creates the right-hand side dictionary from
the one on the left-hand side.

_Write your answer here._

[Hint](../31_Hints/Hints_08_2_02.ipynb)
[Answer](../32_Answers/Answers_08_2_02.ipynb)

#### Exercise 8.2.3

Implement the algorithm to replace `pass` in the function below.
You don't need to add further tests.

In [19]:
%run -i ../m269_util

def invert(original: dict) -> dict:
    """Return the inverted dictionary.

    In both dictionaries, the keys are strings and
    the values are lists of strings.

    Postconditions:
    word1 in output[word2] if and only if word2 in original[word1]
    """
    inverted = dict()
    pass
    return inverted

pt_to_en = {
    'carro': ['car'],
    'andar': ['floor', 'walk'],     # as in 'second floor'
    'chão': ['floor'],              # as in 'wooden floor'
    'saudade': []                   # translation omitted
}

en_to_pt = {
    'car' : ['carro'],
    'walk': ['andar'],
    'floor': ['andar', 'chão']
}

invert_tests = [
    #case,              a_to_b,             inverted
    ('no words',        dict(),             dict()),
    ('pt_to_en',        pt_to_en,           en_to_pt)
]

test(invert, invert_tests)

[Answer](../32_Answers/Answers_08_2_03.ipynb)

#### Optional exercises

The Kattis Guide lists further [problems on dictionaries](https://mwermelinger.github.io/kattis-guide/unordered.html#dictionaries).

⟵ [Previous section](08_1_map.ipynb) | [Up](08-introduction.ipynb) | [Next section](08_3_hash_table.ipynb) ⟶